In [1]:
import xlutils
import databaker
import databaker.constants
from databaker.constants import *
import xypath
import databaker.databakersolo as ds
Dcelllookup = ds.overrides.Dcelllookup
batchcelllookup = ds.overrides.batchcelllookup


In [2]:
tableset = xypath.loader.table_set("gdpq3m2reftables_tcm77-425649.xls", extension='xls')
tabs = list(xypath.loader.get_sheets(tableset, "*"))
print("Table names", [tab.name  for tab in tabs])


Table names ['Content', 'Index', 'A1 AGGREGATES', 'A2 AGGREGATES', 'B1 CVM OUTPUT', 'B2 CVM OUTPUT', 'C1 EXPENDITURE', 'C2 EXPENDITURE', 'D INCOME', 'H1 TRADE', 'H2 TRADE', 'L GVAbp', 'M Alignment adjustments', 'N Financial Year Variables', 'O Selected imp def', 'P GDP per head']


In [3]:
skipcount = {
            'A1 AGGREGATES':1, 'A2 AGGREGATES':1, 'B1 CVM OUTPUT':2, 'B2 CVM OUTPUT':2, 'C1 EXPENDITURE':1, 'C2 EXPENDITURE':1,  
            'D INCOME':1, 'L GVAbp':1, 'M Alignment adjustments':1, 'N Financial Year Variables':1, 'O Selected imp def':1, 
            'P GDP per head':1, 'H1 TRADE':1 ,'H2 TRADE':1}

# Use a dictionary to specifiy the non-percentage measuretype.
mtype = {
            'A1 AGGREGATES':'Number', 'A2 AGGREGATES':'Value', 'B1 CVM OUTPUT':'Number', 'B2 CVM OUTPUT':'Number', 'C1 EXPENDITURE':'Value', 'C2 EXPENDITURE':'Value',  
            'D INCOME':'Value', 'L GVAbp':'Number', 'M Alignment adjustments':'Value', 'N Financial Year Variables':'Value', 'O Selected imp def':'Number', 
            'P GDP per head':'Number', 'H1 TRADE':'Value' ,'H2 TRADE':'Value'}    


conversionsegments = [ ]
for tab in tabs:
    if tab.name == "A1 AGGREGATES":
        anchor = tab.excel_ref('A2').expand(DOWN).is_not_blank().is_not_whitespace().by_index(skipcount[tab.name])
        obs = anchor.fill(RIGHT).expand(DOWN).is_not_blank().is_not_whitespace()
        dimensions = [ 
            (tab.excel_ref('A2').expand(RIGHT).is_not_blank().is_not_whitespace().parent(), "Expenditure", CLOSEST, LEFTRIGHT), 
            (tab.excel_ref('A3').expand(RIGHT).is_not_blank().is_not_whitespace(), "Exp1", DIRECTLY, ABOVE), 
        ]
        #obs = obs - (tab.excel_ref('F1').fill(DOWN) | tab.excel_ref('J1').fill(DOWN))
        #obs = tab.excel_ref('F1') | tab.excel_ref('J1')
        #obs = obs.waffle(anchor.fill(DOWN).is_not_bold().is_not_blank().is_not_whitespace())
        conversionsegments.append((tab, dimensions, obs))
        
# preview the cells in this batch
from databaker.jupybakeutils import dsubsets, tabletohtml, sidewindowhtmldisplay, inlinehtmldisplay
tab, dimensions, segment = conversionsegments[-1]
htmtable = tabletohtml(tab, dsubsets(dimensions, segment))

        
        

In [4]:
#inlinehtmldisplay(htmtable)
sidewindowhtmldisplay("bingo")


TypeError: sidewindowhtmldisplay() takes 0 positional arguments but 1 was given

In [8]:
"""
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "A1 AGGREGATES" None Y
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "A1 AGGREGATES" None Q
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "A2 AGGREGATES" None Y
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "A2 AGGREGATES" None Q
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "B1 CVM OUTPUT" "B2 CVM OUTPUT" Y
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "B1 CVM OUTPUT" "B2 CVM OUTPUT" Q
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "C1 EXPENDITURE" "C2 EXPENDITURE" Y
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "C1 EXPENDITURE" "C2 EXPENDITURE" Q
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "D INCOME" None Y
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "D INCOME" None Q
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "H1 TRADE" "H2 TRADE" Y
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "H1 TRADE" "H2 TRADE" Q
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "L GVAbp" None Y
bake --preview recipe.py gdpq3m2reftables_tcm77-425649.xls "L GVAbp" None Q
"""
DOWN

(0, 1)

In [ ]:

# this does the dereferencing into the list of dicts
tab, dimensions, segment = conversionsegments[-1]
obslist = list(segment.unordered_cells)  # list(segment) otherwise gives bags of one element
dimvalues = [ batchcelllookup(tab, obslist, dimension)  for dimension in dimensions ]
dtuples = zip(*([obslist]+dimvalues))
keys = [OBS] + [ dimension[1]  for dimension in dimensions ]
dvalues = [ dict(zip(keys, dtup))  for dtup in dtuples ]
dvalues[:5]


In [ ]:
allvalues = [ ]
for tab, dimensions, segment in conversionsegments:
    print(tab.name, [d[1]  for d in dimensions])
    for ob in segment:
        v = { OBS:ob.value }
        for header_bag, name, strict, direction in dimensions:
            try:
                v[name] = ds.overrides.Dcelllookup(header_bag, direction, strict, None, ob).value
            except xypath.NoLookupError:
                v[name] = None
                #print("no lookup", header_bag, ob)

In [ ]:
tab, dimensions, segment = conversionsegments[-1]


# convert the segments and handle the messy cases, eg 2010^2 and missing values
allvalues = [ ]
for tab, dimensions, segment in conversionsegments:
    labels = [ OBS ]
    vlist = [ [ ob.value if ob else None  for ob in segment ] ]
    for dimension in dimensions
    for ob in segment:
        v = { OBS:ob.value }

    k = batchcelllookup(tab, segment[:50], dimensions[0])
    print(tab.name, [d[1]  for d in dimensions])
    for ob in segment:
        v = { OBS:ob.value }
        for header_bag, name, strict, direction in dimensions:
            try:
                v[name] = Dcelllookup(header_bag, direction, strict, None, ob).value
            except xypath.NoLookupError:
                v[name] = None
                #print("no lookup", header_bag, ob)
        if isinstance(v[TIME], str):
            v[TIME] = int(v[TIME][:4])
        if isinstance(v[OBS], str):
            continue
        allvalues.append(v)


In [3]:
from IPython.display import display
from IPython.core.display import HTML

# blocked in Chrome??
# How to get javasccript executing in it
#display(HTML('<script>var jj = window.open("http://www.w3schools.com");</script>'))


In [4]:
from IPython.display import display
from IPython.core.display import HTML

def sidewindowhtmldisplay(htm):
    sJavascript = '''
<b>Spawning "sidewin" window</b> 
<script type="text/Javascript">
var Dcheck = "something"; 
var sidewin = "sssomething"; 
sidewin = window.open("", "abc123", "toolbar=no,location=no,,status=no,menubar=no,scrollbars=yes,resizable=yes,width=780,height=200,top=200,left=200"); 
// use backtick for multi-line string
var sHtml = `
%s
`;
if (sidewin)
    sidewin.document.body.innerHTML = "<h1>Hello</h1> " + sHtml;
else
    alert("sidewindow didn't work"); 
</script>
'''
    display(HTML(sJavascript % htm))

#sidewindowhtmldisplay(htmtable)


In [14]:
sidewin


1